In [55]:
using JuMP, Ipopt, SparseArrays, LinearAlgebra, NamedArrays, DelimitedFiles, DataFrames, CSV, Statistics, PyPlot

# Include necessary functions
include("Functions/Ybus.jl");         # Function to calculate Ybus matrix (G,B)
include("Functions/Conventional.jl");   # Function to define constraints (polar)
include("Functions/Power_flow.jl");    # Function to calculate power flow
include("Functions/Cost.jl");          # Function to define cost (polar)
include("Functions/Misc.jl");                # Function to define start point,map input,print output
include("Functions/Parse_gld_1day.jl");      # Function to parse GridLabD system data
include("Functions/Fixed_point.jl");     # Function to implement fixed-point method
include("Functions/BFS.jl");             # Function to implement BFS method
include("Functions/Dist3Flow.jl");       # Function to implement Dist3Flow method
include("Functions/Active_constr.jl");   # Function to find active constraints

# Load test case
global case_name = "case13_test" 
#global case_name = "GC-12.47-1" 
# global case_name = "R1-12.47-3"  
#global case_name = "turkey_solar"
data_file = open(string("data/",case_name,"/data.xml")); data = readlines(data_file)
z_file = open(string("data/",case_name,"/z_dump.xml")); z_dump = readlines(z_file)
global PV_en = 0;   # 0-exclude, 1-include ZIP loads/PV inverters
    if PV_en == 1
        house_sch = CSV.read(string("House_",case_name,"_profile.csv"),header=false,type=Float64,DataFrame)
        global house_sch = convert(Matrix,house_sch);
        PV_sch = CSV.read(string("PV_",case_name,"_profile.csv"),header=false,type=Float64,DataFrame)
        global PV_sch = convert(Matrix,PV_sch);
    end
# global VUF_init = readdlm(string("VUF_",case_name,"_init.csv"), ',')/100
# global QgY_init = readdlm(string("QgY0_",case_name,"_base.csv"), ',');
# global QgY_init = readdlm("QgY0_R1-12.47-1_freq60_delay60.csv", ',');

## Other inputs
    global count_sch = 1;  # Time instant to run OPF
#     global crit_node = ["632"]
global obj_op = 2
    # global crit_node = ["l_49"]
    # global crit_node = ["R1-12-47-1_node_17"]
    # global crit_node = ["R1-12-47-3_node_1"]
    global crit_node = "all"
    global PV_rat = 0.035;
    global Q_pen = 0*1e-5; global ΔV_pen = 0*1e-2;

        
## Parse system data
parse_gld(crit_node); global M = 1e3

## Create Y_bus and connection matrices
Ybus(); conn_bgl();

   # Initialize values
   start_point(4); global Vm0 = s0[1:nb_nph]; global θ0 = s0[nb_nph+1:2*nb_nph];
   global Vd0 = Vm0.*cos.(θ0); global Vq0 = Vm0.*sin.(θ0); global QgY0 = zeros(ngy_inv);
   stat = "LOCALLY_SOLVED";
   bus_ID

par_det = 8×1 Named Adjoint{Int64, Matrix{Int64}}
Par ╲ Val │ :Value
──────────┼───────
nb        │      7
ngy       │      1
nly       │      2
nld       │      1
nb_nph    │     19
ngy_nph   │      3
nly_nph   │      5
nld_nph   │      1
-------------------


7-element Vector{String}:
 "n633"
 "n630"
 "n632"
 "n650"
 "l634"
 "l645"
 "l646"

In [85]:
## KVL constraints
count2 = 1;
for i=1:nbr
    j = Int(branch[i,1]); jj = Int(sum(bus_count[1:j-1])+1):Int(sum(bus_count[1:j]))
    k = Int(branch[i,2]); kk = Int(sum(bus_count[1:k-1])+1):Int(sum(bus_count[1:k]))
    nz = findmin([length(jj) length(kk)])[1]
    jjjj = zeros(nz); kkkk = zeros(nz); num_count=1
    for jjj=jj
        for kkk=kk
            if bus_ϕ[jjj] == bus_ϕ[kkk]
                jjjj[num_count] = jjj; kkkk[num_count] = kkk;
                num_count=num_count+1
            end
        end
    end
    jj = Int.(jjjj); kk = Int.(kkkk); 
    Zbr = -pinv(Y[jj,kk]); Rbr = real(Zbr); Xbr = imag(Zbr);
    if tf_branch[i,3] == 8
        if bus_ph[k] == "AN"
            vr_tap = 1 + 0.00625*tf_branch[vr_idx[count2],6]
        elseif bus_ph[k] == "BN"
            vr_tap = 1 + 0.00625*tf_branch[vr_idx[count2],7]
        elseif bus_ph[k] == "CN"
            vr_tap = 1 + 0.00625*tf_branch[vr_idx[count2],8]
        elseif bus_ph[k] == "ABN" || bus_ph[k] == "BAN"
            vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count2],6:7])
        elseif bus_ph[k] == "BCN" || bus_ph[k] == "CBN"
            vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count2],7:8])
        elseif bus_ph[k] == "ACN" || bus_ph[k] == "CAN"
            vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count2],6:2:8])
        else
            vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count2],6:8])
        end
        for (idx_r,jjj) in enumerate(jj)
            kkk = kk[idx_r]; 
            # @NLconstraint(m, Vm[kkk]*cos(θ[kkk]) == vr_tap*(Vm[jjj]*cos(θ[jjj]) - sum(Rbr[idx_r,idx_c]*Ibus_d[val] for (idx_c,val) in enumerate(kk)) + sum(Xbr[idx_r,idx_c]*Ibus_q[val] for (idx_c,val) in enumerate(kk))) )
            # @NLconstraint(m, Vm[kkk]*sin(θ[kkk]) == vr_tap*(Vm[jjj]*sin(θ[jjj]) - sum(Rbr[idx_r,idx_c]*Ibus_q[val] for (idx_c,val) in enumerate(kk)) - sum(Xbr[idx_r,idx_c]*Ibus_d[val] for (idx_c,val) in enumerate(kk))) )
        end
        count2 = count2 + 1;
    else
        for (idx_r,jjj) in enumerate(jj)
            kkk = kk[idx_r]; 
            # @NLconstraint(m, Vm[kkk]*cos(θ[kkk]) == (Vm[jjj]*cos(θ[jjj]) - sum(Rbr[idx_r,idx_c]*Ibus_d[val] for (idx_c,val) in enumerate(kk)) + sum(Xbr[idx_r,idx_c]*Ibus_q[val] for (idx_c,val) in enumerate(kk))) )
            # @NLconstraint(m, Vm[kkk]*sin(θ[kkk]) == (Vm[jjj]*sin(θ[jjj]) - sum(Rbr[idx_r,idx_c]*Ibus_q[val] for (idx_c,val) in enumerate(kk)) - sum(Xbr[idx_r,idx_c]*Ibus_d[val] for (idx_c,val) in enumerate(kk))) )
        end
     end
end

true

In [62]:
global m = Model(optimizer_with_attributes(Ipopt.Optimizer,"warm_start_init_point"=>"yes","print_level"=>0, "tol"=>1e-7));
 @variable(m, Ibus_d[i=1:nb_nph], start = 0 ); 
             @variable(m, Ibus_q[i=1:nb_nph], start = 0 );


In [8]:
## Backward sweep
V0 = Vm0.*exp.(im*θ0); S_inj = Dload+Yload; count3 = 1;
global Id_inj = @variable(m,[i=1:nb_nph] ); global Iq_inj = @variable(m,[i=1:nb_nph] );
@constraint(m, Id_inj .== ((real(YDload)-Cbg*PgY).*cos.(θ0) + (imag(YDload)-Cbg*QgY).*sin.(θ0))./Vm0 );
@constraint(m, Id_inj .== ((real(YDload)-Cbg*PgY).*sin.(θ0) - (imag(YDload)-Cbg*QgY).*cos.(θ0))./Vm0 );
# global Id_inj = @expression(m,[i=1:nb_nph], ((real(YDload)-Cbg*PgY).*cos.(θ0) + (imag(YDload)-Cbg*QgY).*sin.(θ0))./Vm0 );
# global Iq_inj = @expression(m,[i=1:nb_nph], ((real(YDload)-Cbg*PgY).*sin.(θ0) - (imag(YDload)-Cbg*QgY).*cos.(θ0))./Vm0 );
Ibus_d = @expression(m,[i=1:nb_nph], 0*Vd[1]); Ibus_q = @expression(m, [i=1:nb_nph], 0*Vq[1]);
# global Ibus_d = @variable(m,[i=1:nb_nph]); global Ibus_q = @variable(m, [i=1:nb_nph]);
for i=1:nbr
    j = Int(line_prove[nbr-i+1,2]); jj = Int(sum(bus_count[1:j-1])+1):Int(sum(bus_count[1:j]))
    k = Int(line_prove[nbr-i+1,1]); kk = Int(sum(bus_count[1:k-1])+1):Int(sum(bus_count[1:k]))
    nz = findmin([length(jj) length(kk)])[1]
    jjjj = zeros(nz); kkkk = zeros(nz); num_count=1
    for jjj=jj
        for kkk=kk
            if bus_ϕ[jjj] == bus_ϕ[kkk]
                jjjj[num_count] = jjj; kkkk[num_count] = kkk;
                num_count=num_count+1
            end
        end
    end
    jj = Int.(jjjj); kk = Int.(kkkk)
    if tf_branch[Int(line_prove[nbr-i+1,4]),3] == 8
        if bus_ph[k] == "AN"
            vr_tap = 1 + 0.00625*tf_branch[vr_idx[count3],6]
        elseif bus_ph[k] == "BN"
            vr_tap = 1 + 0.00625*tf_branch[vr_idx[count3],7]
        elseif bus_ph[k] == "CN"
            vr_tap = 1 + 0.00625*tf_branch[vr_idx[count3],8]
        elseif bus_ph[k] == "ABN" || bus_ph[k] == "BAN"
            vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count3],6:7])
        elseif bus_ph[k] == "BCN" || bus_ph[k] == "CBN"
            vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count3],7:8])
        elseif bus_ph[k] == "ACN" || bus_ph[k] == "CAN"
            vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count3],6:2:8])
        else
            vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count3],6:8])
        end
        @constraint(m, Ibus_d[kk] .== vr_tap*(Ibus_d[kk] + Id_inj[kk]) ); 
        @constraint(m, Ibus_q[kk] .== vr_tap*(Ibus_q[kk] + Iq_inj[kk]) );
        count3 = count3+1;
    else
        @constraint(m, Ibus_d[kk] .== Ibus_d[kk] + Id_inj[kk] ); 
        @constraint(m, Ibus_q[kk] .== Ibus_q[kk] + Iq_inj[kk] );
    end
    @constraint(m, Ibus_d[jj] .== Ibus_d[kk] + Ibus_d[jj] ); 
    @constraint(m, Ibus_q[jj] .== Ibus_q[kk] + Ibus_q[jj] );
end

In [9]:
 ## Forward sweep
 count2 = 1;
 for i=1:nbr
     j = Int(line_prove[i,2]); jj = Int(sum(bus_count[1:j-1])+1):Int(sum(bus_count[1:j]))
     k = Int(line_prove[i,1]); kk = Int(sum(bus_count[1:k-1])+1):Int(sum(bus_count[1:k]))
     nz = findmin([length(jj) length(kk)])[1]
     jjjj = zeros(nz); kkkk = zeros(nz); num_count=1
     for jjj=jj
         for kkk=kk
             if bus_ϕ[jjj] == bus_ϕ[kkk]
                 jjjj[num_count] = jjj; kkkk[num_count] = kkk;
                 num_count=num_count+1
             end
         end
     end
     jj = Int.(jjjj); kk = Int.(kkkk); Zbr = -pinv(Y[jj,kk]);
     if tf_branch[Int(line_prove[i,4]),3] == 8
         if bus_ph[k] == "AN"
             vr_tap = 1 + 0.00625*tf_branch[vr_idx[count2],6]
         elseif bus_ph[k] == "BN"
             vr_tap = 1 + 0.00625*tf_branch[vr_idx[count2],7]
         elseif bus_ph[k] == "CN"
             vr_tap = 1 + 0.00625*tf_branch[vr_idx[count2],8]
         elseif bus_ph[k] == "ABN" || bus_ph[k] == "BAN"
             vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count2],6:7])
         elseif bus_ph[k] == "BCN" || bus_ph[k] == "CBN"
             vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count2],7:8])
         elseif bus_ph[k] == "ACN" || bus_ph[k] == "CAN"
             vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count2],6:2:8])
         else
             vr_tap = Diagonal(1 .+ 0.00625*tf_branch[vr_idx[count2],6:8])
         end
         @constraint(m, Vd[kk] .== vr_tap*Vd[jj] - real(Zbr)*Ibus_d[kk] )
         @constraint(m,  Vq[kk] .== vr_tap*Vq[jj] - real(Zbr)*Ibus_q[kk] )
         count2 = count2 + 1;
     else
         @constraint(m, Vd[kk] .== Vd[jj] - real(Zbr)*Ibus_d[kk] + imag(Zbr)*Ibus_q[kk] )
         @constraint(m, Vq[kk] .== Vq[jj] - real(Zbr)*Ibus_q[kk] - imag(Zbr)*Ibus_d[kk] )
     end
     # Vm_jj = Vm1[jj].*exp.(im*θ1[jj]); ii = Int(line_prove[i,4])
     # if bus_ph[k] == "AN" || bus_ph[k] == "AS" || bus_ph[k] == "A"
     #     Vm_kk = A_mat[1,1,ii]*Vm_jj - B_mat[1,1,ii]*Ibr[count1:count1+length(kk)-1]
     # elseif bus_ph[k] == "BN" || bus_ph[k] == "BS" || bus_ph[k] == "B"
     #     Vm_kk = A_mat[2,2,ii]*Vm_jj - B_mat[2,2,ii]*Ibr[count1:count1+length(kk)-1]
     # elseif bus_ph[k] == "CN" || bus_ph[k] == "CS" || bus_ph[k] == "C"
     #     Vm_kk = A_mat[3,3,ii]*Vm_jj - B_mat[3,3,ii]*Ibr[count1:count1+length(kk)-1]
     # elseif bus_ph[k] == "ABN" || bus_ph[k] == "ABD" || bus_ph[k] == "BAN" || bus_ph[k] == "BAD" || bus_ph[k] == "BA" || bus_ph[k] == "AB"
     #     Vm_kk = A_mat[1:2,1:2,ii]*Vm_jj - B_mat[1:2,1:2,ii]*Ibr[count1:count1+length(kk)-1]
     # elseif bus_ph[k] == "BCN" || bus_ph[k] == "BCD" || bus_ph[k] == "CBN"  || bus_ph[k] == "CBD" || bus_ph[k] == "CB" || bus_ph[k] == "BC"
     #     Vm_kk = A_mat[2:3,2:3,ii]*Vm_jj - B_mat[2:3,2:3,ii]*Ibr[count1:count1+length(kk)-1]
     # elseif bus_ph[k] == "ACN" || bus_ph[k] == "ACD" || bus_ph[k] == "CAN" || bus_ph[k] == "CAD" || bus_ph[k] == "CA" || bus_ph[k] == "AC"
     #     Vm_kk = A_mat[1:2:3,1:2:3,ii]*Vm_jj - B_mat[1:2:3,1:2:3,ii]*Ibr[count1:count1+length(kk)-1]
     # else
     #     Vm_kk = A_mat[:,:,ii]*Vm_jj - B_mat[:,:,ii]*Ibr[count1:count1+length(kk)-1]
     # end
 end

 ## Define cost
 cost_rect(Vll);

 ## Solve and update solution
 @time status = optimize!(m); sts = string(termination_status(m))

  0.001199 seconds (2.12 k allocations: 279.234 KiB)


"INVALID_MODEL"